<a href="https://colab.research.google.com/github/pablomerizalde/CO0371-Analitica-de-negocios/blob/main/NN/03%20-%20credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pandas
!pip3 install openpyxl
!pip3 install scikit-learn

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Mapa para convertir las decisiones a valores numéricos, en mayusculas porque es una constante
DECISION_MAP = {'Aprobado':1,'Negado':0}

# Importamos los datos

In [3]:
# Cargamos los datos crudos de los aprobados
#nxl='./../datos/DatosRiesgoCredito.xlsx.xlsx'
nxl= 'https://raw.githubusercontent.com/pablomerizalde/CO0371-Analitica-de-negocios/main/datos/DatosRiesgoCredito.xlsx'

df_crudo_1:pd.DataFrame = pd.read_excel(nxl,sheet_name='CreditosAprobados')


print(df_crudo_1.shape)
df_crudo_1.head(5)

(198, 22)


,Unnamed: 0,Edad,Genero,Estado civil,Hijos,Perscargo,Nivel Renta,Nivel de estudios,Contratación,Ingresos (U$),...,Monto n(U$),Plazo,Cuota (U$),ConoCliente,Garantia,Patrimonio,Preaprobación,Decisión,EndeuIndi,Puntaje
0,1,22,Mujer,Soltero,0,0,2,Técnico,Indefinido,271.142857,...,457.142857,24,11.694286,Buenas Referencias,Libranza Establecida,Sin Patrimonio,1,Aprobado,0,27.0
1,2,57,Mujer,Casado,2,0,4,Bachiller,Ninguno,516.000000,...,1428.571429,36,61.123429,Buena Experiencia,Sin Codeudor,Prop.sin Hipoteca,1,Aprobado,0,63.0
2,3,21,Mujer,Casado,0,0,2,Bachiller,Fijo,233.142857,...,1771.428571,36,70.619143,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,0,Aprobado,35,NaN
3,4,23,Mujer,Soltero,0,0,2,Técnico,Indefinido,166.571429,...,371.428571,12,35.230000,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,1,Aprobado,0,17.0
4,5,22,Mujer,Soltero,0,0,2,Técnico,Fijo,214.285714,...,371.428571,18,24.993429,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,1,Aprobado,0,20.0


In [5]:
# Cargamos los datos crudos de los negados
df_crudo_2:pd.DataFrame = pd.read_excel(nxl,sheet_name='CreditosNegados')
print(df_crudo_2.shape)
df_crudo_2.head(5)

(202, 22)


,Unnamed: 0,Edad,Genero,Estado civil,Hijos,Perscargo,Nivel Renta,Nivel de estudios,Contratación,Ingresos (U$),...,Monto n(U$),Plazo,Cuota (U$),ConoCliente,Garantia,Patrimonio,Preaprobación,Decisión,EndeuIndi,Puntaje
0,1,43,Mujer,Casado,0,0,3,Profesional,Indefinido,342.857143,...,76.857143,3,26.805143,Buenas Referencias,Con Codeudor(es),Prop.con Hipoteca,1,Negado,54,NaN
1,2,44,Mujer,Casado,3,0,5,Profesional,Indefinido,445.888286,...,857.142857,36,35.602857,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,1,Negado,18,NaN
2,3,72,Hombre,Casado,8,1,4,Bachiller,Ninguno,643.002857,...,2857.142857,48,100.101429,Sin Referencias,Con Codeudor(es),Prop.y Vehìculo,1,Negado,0,86.0
3,4,69,Hombre,Casado,4,0,3,Bachiller,Ninguno,149.174000,...,228.571429,18,15.733143,Buenas Referencias,Con Codeudor(es),Sin Patrimonio,0,Negado,0,60.0
4,5,64,Hombre,Unión Libre,4,4,3,Bachiller,Ninguno,451.274571,...,342.857143,18,23.599429,Buenas Referencias,Sin Codeudor,Sin Patrimonio,1,Negado,0,61.0


# Preprocesamiento

In [6]:
# Unimos los df
df_crudo = pd.concat([df_crudo_1,df_crudo_2],ignore_index=True)
# Cambiamos los valores de la columna 'Decisión' a 1 y 0
df_crudo['Decisión'] = df_crudo['Decisión'].map(DECISION_MAP)
# Eliminamos filas con valores nulos
df_crudo.dropna(inplace=True)
# Revisamos el tamañ del df final
print(df_crudo.shape)

(217, 22)


In [7]:
# Seleccionamos solo las columnas numéricas y eliminamos las que no se usarán y reordenamos aleatoriamente el df
df_numericas = df_crudo.select_dtypes(include=['number']).drop(columns=['Unnamed: 0', 'EndeuIndi']).sample(217)
print(df_numericas.shape)
df_numericas.head(5)

(217, 12)


,Edad,Hijos,Perscargo,Nivel Renta,Ingresos (U$),Egresos (U$),Monto n(U$),Plazo,Cuota (U$),Preaprobación,Decisión,Puntaje
34,26,0,0,3,437.203714,68.000000,2857.142857,48,95.053714,1,1,51.0
203,42,2,2,2,160.428571,0.000000,714.285714,24,38.194286,1,0,45.0
359,42,3,3,2,1857.142857,1819.714286,228.571429,6,40.874857,1,0,31.0
344,61,5,2,1,147.714286,29.714286,314.285714,18,21.399143,1,0,46.0
111,43,0,0,5,714.285714,256.571429,2857.142857,60,177.306000,0,1,80.0


In [8]:
# Escalamos las variables numéricas entre 0 y 1 usando MinMaxScaler
scaler = MinMaxScaler()
# Separamos las variables independientes
X = df_numericas.drop(columns=['Decisión','Preaprobación','Puntaje'])
X_scaled = scaler.fit_transform(X)
X_scaled[1]

array([0.38888889, 0.25      , 0.25      , 0.2       , 0.02874597,
       0.        , 0.08737864, 0.36842105, 0.09252114])

In [9]:
# Separamos la variable dependiente
y = df_numericas['Decisión'].values
y[1]

np.int64(0)

In [10]:
import torch
#Componentes para Neural network NN
import torch.nn as nn
#Componentes para entrenar (optimizar) las NN
import torch.optim as optim

# Convertimos a tensores de PyTorch con el tipo de dato float32
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
# Dividimos en datos de entrenamiento y prueba
x_test, y_test = X_tensor[:20], y_tensor[:20]
x_train, y_train = X_tensor[20:], y_tensor[20:]
# Verificamos las dimensiones de los conjuntos
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([197, 9]),
 torch.Size([197, 1]),
 torch.Size([20, 9]),
 torch.Size([20, 1]))

In [11]:
# Definimos el modelo de la red neuronal con una capa oculta de 10 neuronas y función de activación ReLU con entrada de 9 características
model = nn.Sequential(
    nn.Linear(9, 10),
    nn.ReLU(),
    nn.Linear(10, 1),
)
# Definimos la función de pérdida y el optimizador
loss_function = nn.MSELoss()
# Definimos el optimizador gradiente descendiente estocástico (SGD) con una tasa de aprendizaje de 0.01
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [12]:
# Función para entrenar el modelo
def train_model(model, X, y, loss_function, optimizer, epochs=1000):
    '''Entrena el modelo de red neuronal.
    Parámetros:
    model: el modelo de red neuronal a entrenar
    X: datos de entrada (características)
    y: datos de salida (etiquetas)
    loss_function: función de pérdida para evaluar el rendimiento del modelo
    optimizer: optimizador para actualizar los pesos del modelo
    epochs: número de épocas para entrenar el modelo 1000 por defecto
    '''
    # se itera sobre el número de épocas definido
    for epoch in range(epochs):
        # Reiniciamos los gradientes del optimizador a cero antes de cada pasada
        optimizer.zero_grad()
        # Hacemos las predicciones con el modelo
        predictions = model(X)
        # Calculamos la pérdida entre las predicciones y los valores reales
        loss = loss_function(predictions, y)
        # Calculamos los gradientes de la pérdida con respecto a los parámetros del modelo
        loss.backward()
        # actualizamos los parámetros del modelo usando el optimizador
        optimizer.step()
        # Imprimimos el error cada 100 épocas
        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}: Loss = {loss.item()}")
# Entrenamos el modelo
train_model(model, x_train, y_train, loss_function, optimizer, epochs=1500)
# Evaluamos el modelo en los datos de prueba
# apagamos el cálculo de gradientes para evaluación esto nos permite ahorrar memoria y mejorar la velocidad
with torch.no_grad():
    # Hacemos predicciones en los datos de prueba
    y_pred = model(x_test)
# Calculamos la pérdida en los datos de prueba
loss = loss_function(y_pred, y_test)
print(f"Test Loss: {loss.item()}")

Epoch 100: Loss = 0.22657741606235504
Epoch 200: Loss = 0.22002820670604706
Epoch 300: Loss = 0.2126544862985611
Epoch 400: Loss = 0.20395050942897797
Epoch 500: Loss = 0.19329993426799774
Epoch 600: Loss = 0.17927710711956024
Epoch 700: Loss = 0.16463986039161682
Epoch 800: Loss = 0.15049584209918976
Epoch 900: Loss = 0.13762910664081573
Epoch 1000: Loss = 0.1263001263141632
Epoch 1100: Loss = 0.1170937791466713
Epoch 1200: Loss = 0.11001428961753845
Epoch 1300: Loss = 0.10466232150793076
Epoch 1400: Loss = 0.10053494572639465
Epoch 1500: Loss = 0.0972018912434578
Test Loss: 0.06883543729782104


In [13]:
# Probamos el modelo con un dato único
test_unique = df_numericas.iloc[0,:].drop(labels=['Decisión','Preaprobación','Puntaje'])
# Escalamos el dato único
test_scaled = scaler.transform([test_unique])
# Convertimos el dato escalado a un tensor de PyTorch
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)
# Hacemos la predicción
with torch.no_grad():
    prediction = model(test_tensor)
print(f"Prediction for input {test_unique} \nPredicción: {prediction.item()}")

Prediction for input Edad               26.000000
Hijos               0.000000
Perscargo           0.000000
Nivel Renta         3.000000
Ingresos (U$)     437.203714
Egresos (U$)       68.000000
Monto n(U$)      2857.142857
Plazo              48.000000
Cuota (U$)         95.053714
Name: 34, dtype: float64 
Predicción: 0.9995241165161133


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [14]:
# Probamos el modelo con otro dato único
test_unique = df_numericas.iloc[2,:].drop(labels=['Decisión','Preaprobación','Puntaje'])
# Escalamos el dato único
test_scaled = scaler.transform([test_unique])
# Convertimos el dato escalado a un tensor de PyTorch
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)
# Hacemos la predicción
with torch.no_grad():
    prediction = model(test_tensor)
print(f"Prediction for input {test_unique} \nPredicción: {prediction.item()}")

Prediction for input Edad               42.000000
Hijos               3.000000
Perscargo           3.000000
Nivel Renta         2.000000
Ingresos (U$)    1857.142857
Egresos (U$)     1819.714286
Monto n(U$)       228.571429
Plazo               6.000000
Cuota (U$)         40.874857
Name: 359, dtype: float64 
Predicción: -0.005628049373626709


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
